In [10]:
import pandas as pd

In [11]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')


In [12]:
print(" =============== TRAIN ================")
print(train.info())
print(train.isna().sum())
print(" ======================================")
print(" =============== TEST ================")
print(test.info())
print(test.isna().sum())
print(" ======================================")

 =============== TRAIN ================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ 

### Data preprocessing  

In [13]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [14]:
train.columns

Index(['id', 'age', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [15]:
# Binary categorical features mapping
binary_map = {'yes' : 1, 'no' : 0}
binary_columns = ['default', 'housing', 'loan']

for col in binary_columns:
    train[col] = train[col].map(binary_map)
    test[col] = test[col].map(binary_map)

# Ordinal encoding for date-related features
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

ord_enc = OrdinalEncoder(categories=[month_order])
train[['month']] = ord_enc.fit_transform(train[['month']]).astype(int)
test[['month']] = ord_enc.fit_transform(test[['month']]).astype(int)


# One-Hot encode categorical features (excluding already encoded ones)






In [16]:
categorical_cols = ['job', 'marital', 'education', 'contact', 'poutcome']

# Fill missing values in categorical columns
# train[categorical_cols] = train[categorical_cols].fillna("missing")
# test[categorical_cols] = test[categorical_cols].fillna("missing")

# Fit the encoder on train data only
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
encoder.fit(train[categorical_cols])

# Transform train and test data
encoded_train = encoder.transform(train[categorical_cols])
encoded_test = encoder.transform(test[categorical_cols])

# Convert to DataFrame
encoded_train_df = pd.DataFrame(encoded_train.toarray().astype(int), columns=encoder.get_feature_names_out(categorical_cols))
encoded_test_df = pd.DataFrame(encoded_test.toarray().astype(int), columns=encoder.get_feature_names_out(categorical_cols))

# Concatenate the encoded features with the original data
train = pd.concat([train.reset_index(drop=True), encoded_train_df], axis=1).drop(columns=categorical_cols)
test = pd.concat([test.reset_index(drop=True), encoded_test_df], axis=1).drop(columns=categorical_cols)

In [17]:
# Show the final dataframe structure
print("DataFrame shape after encoding:", train.shape)
print("Sample data:\n", train.head())

DataFrame shape after encoding: (750000, 34)
Sample data:
    id  age  default  balance  housing  loan  day  month  duration  campaign  \
0   0   42        0        7        0     0   25      7       117         3   
1   1   38        0      514        0     0   18      5       185         1   
2   2   36        0      602        1     0   14      4       111         2   
3   3   27        0       34        1     0   28      4        10         2   
4   4   26        0      889        1     0    3      1       902         1   

   ...  marital_married  marital_single  education_secondary  \
0  ...                1               0                    1   
1  ...                1               0                    1   
2  ...                1               0                    1   
3  ...                0               1                    1   
4  ...                1               0                    1   

   education_tertiary  education_unknown  contact_telephone  contact_unknown  \
0